# Importing Data from Kaggle

https://www.kaggle.com/general/74235 
https://www.youtube.com/watch?v=57N1g8k2Hwc&ab_channel=ShriramVasudevan


In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/muratkokludataset/acoustic-extinguisher-fire-dataset                  Acoustic Extinguisher Fire Dataset               621KB  2022-04-02 22:59:36           1064        716  0.9375           
datasets/muratkokludataset/rice-image-dataset                                  Rice Image Dataset                               219MB  2022-04-03 02:12:00           1173        661  0.875            
datasets/mdmahmudulhasansuzan/students-adaptability-level-in-online-education  Students Adaptability Level in Online Education    6KB  2022-04-16 04:46:28           4186        103  1.0              


Data importing

In [ ]:
! kaggle competitions download -c "ieee-fraud-detection"

 94% 111M/118M [00:00<00:00, 170MB/s] 
100% 118M/118M [00:00<00:00, 138MB/s]


In [ ]:
!unzip ieee-fraud-detection.zip

Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


# Importing Libs


In [ ]:
import numpy as np
import pandas as pd

import math
import os
import random
import re
import sys

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import matplotlib as mpl

# Code


#Identity Table

Variables in this table are identity information like 
* network connection information (IP, ISP, Proxy, etc)  
* digital signature (UA/browser/os/version, etc) associated with transactions.

They're collected by Vesta’s fraud protection system and digital security partners. (The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

###Categorical Features:
* DeviceType
* DeviceInfo
* id_12 - id_38

In [ ]:
##original data
data = pd.read_csv('/content/train_identity.csv',nrows=4000)

data.count()
len(data.columns)
col_labels= data.columns
data.head(3)

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows


In [ ]:
print(col_labels)
data.ndim
data.shape

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')


(4000, 41)

In [ ]:
data.info(verbose=False,memory_usage="deep")
#data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 41 entries, TransactionID to DeviceInfo
dtypes: float64(23), int64(1), object(17)
memory usage: 4.4 MB


In [ ]:
sample= pd.read_csv("/content/sample_submission.csv")
sample.head(1)

,TransactionID,isFraud
0,3663549,0.5


#Transaction Table
* TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
* TransactionAMT: transaction payment amount in USD
* ProductCD: product code, the product for each transaction
* card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
* addr: address
* dist: distance
* P_ and (R__) emaildomain: purchaser and recipient email domain
* C1-C14: counting, such as how many addresses are found to be  associated with the payment card, etc. The actual meaning is masked.
* D1-D15: timedelta, such as days between previous transaction, etc.
* M1-M9: match, such as names on card and address, etc.
* Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

###Categorical Features:
* ProductCD
* card1 - card6
* addr1, addr2
* P_emaildomain
* R_emaildomain
* M1 - M9


In [ ]:
trans= pd.read_csv("/content/train_transaction.csv",nrows=4000)
trans.head(11)
trans_labels=trans.columns
print(trans_labels)

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)


Replacing 'NaN' values with most frequent values for both dataframes

https://www.youtube.com/watch?v=8p6XaQSIFpY&ab_channel=sentdex 

In [ ]:
dataset_1=data
dataset_2=trans

##adding label to dataset 1

dataset_1=dataset_1.merge(dataset_2[['TransactionID','isFraud']], how = 'left')



dataset_1 = dataset_1.fillna(dataset_1.mode().iloc[0])
dataset_2 = dataset_2.fillna(dataset_2.mode().iloc[0])


In [ ]:
def coverting_data(df):
  header=df.columns.values
  tmp=df.columns.values
  for column in header:
    text_to_digit={}
    def covert_to_int(value):
      return text_to_digit[value]

    if df[column].dtype != np.int64 and df[column].dtype != np.float64:
      #print("non numbric col found",column)
      column_contents = df[column].values.tolist() #or use unique function and then set
    #  cl=df[column].unique()
      unique_ele=set(column_contents)
      #print(unique_ele,set(cl))
      x=0
      for unique_val in unique_ele:
        if unique_val not in text_to_digit:
          text_to_digit[unique_val] = x
          x+=1
      
      df[column] = list(map(covert_to_int,df[column]))
    

  return df


#h,t=
dataset_1=coverting_data(dataset_1)
dataset_2=coverting_data(dataset_2)

#print(h,t)
#dataframe


In [ ]:
dataset_1['DeviceInfo'].unique()

array([225, 139,  31, 250, 253,  35, 273,  62, 188,  27, 147, 263,  11,
        25,  68, 101, 281,  89, 247,  34, 266,  72, 280, 267, 158, 146,
        36,  79,  22, 230,  78,  41,  61, 196,  50,  69, 105, 184,  39,
       126, 245, 213, 240, 148, 137,  86,  45,  24, 131, 144,  83,  23,
       238, 255, 234, 180, 222,   7, 221,  48, 186, 201, 145, 212,  70,
        65,  32, 193, 277,  63,  84,  42, 134, 169,   2, 111,   4, 195,
       102, 108,  15,  29, 254, 223, 124,  96, 150, 154, 274, 236, 275,
        80,  64, 170, 194,  16, 100, 256,  13, 209,  12,  90, 268, 241,
       152, 200,  91, 120,  21,  77,  14,  98,  92,   8,  20, 127, 257,
       123, 248, 232, 149, 214, 249, 142, 119, 269,  17, 153, 227, 114,
        19, 106, 191, 270,  55,  71, 157, 217, 261, 156, 202, 165, 107,
        95, 264, 189,  37,  87, 135, 181, 129,  49, 197, 228, 262, 121,
        46, 224, 151, 122, 199, 233,  44, 118, 112, 167,  54, 182,  51,
       177,  97,  18, 132,  56, 258, 161, 246, 183, 128, 133, 25

Merging the 2 dataframe in one with respect to **"TransationID"**

In [ ]:
dataset_3 = pd.merge(dataset_2, dataset_1, how='left', on = ['TransactionID','isFraud'])
dataset_3.shape

(4000, 434)

In [ ]:

dataset_3 = dataset_3.fillna(dataset_3.mode().iloc[0])
dataset_3.head(3)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,1,13926,194.0,150.0,2,142.0,...,20.0,24.0,9.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0
1,2987001,0,86401,29.0,1,2755,404.0,150.0,1,102.0,...,20.0,24.0,9.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0
2,2987002,0,86469,59.0,1,4663,490.0,150.0,3,166.0,...,20.0,24.0,9.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0


## Note

Model would be tested by providing a 'ID' and for that 'ID' column fraud would be given.

In [ ]:
#trans_dataset_labels=trans['isFraud']
#combined_dataset_labels=df_merged_table['isFraud']
#data_with_label= df_merged_table

In [ ]:
dataset_2.head(12)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.500,1,13926,194.0,150.0,2,142.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2987001,0,86401,29.000,1,2755,404.0,150.0,1,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,0,86469,59.000,1,4663,490.0,150.0,3,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,0,86499,50.000,1,18132,567.0,150.0,1,117.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2987004,0,86506,50.000,2,4497,514.0,150.0,1,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2987005,0,86510,49.000,1,5937,555.0,150.0,3,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2987006,0,86522,159.000,1,12308,360.0,150.0,3,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2987007,0,86529,422.500,1,12695,490.0,150.0,3,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2987008,0,86535,15.000,2,2803,100.0,150.0,3,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2987009,0,86536,117.000,1,17399,111.0,150.0,1,224.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dataset_3.head(2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,1,13926,194.0,150.0,2,142.0,...,20.0,24.0,9.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0
1,2987001,0,86401,29.0,1,2755,404.0,150.0,1,102.0,...,20.0,24.0,9.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0


# Class distribution

In [ ]:
print(dataset_1['isFraud'].value_counts())
print(dataset_2['isFraud'].value_counts())
print(dataset_3['isFraud'].value_counts())

0.0    3967
1.0      33
Name: isFraud, dtype: int64
0    3917
1      83
Name: isFraud, dtype: int64
0    3917
1      83
Name: isFraud, dtype: int64


# Trying out SVM

In [ ]:
# from sklearn.model_selection import train_test_split
def data_splitting(dataset,testset_size=0.2,randomState=1):

  ##seperating labels for data 
  X= dataset_1.drop('isFraud', axis = 1).values
  y= dataset_1['isFraud'].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
  #training_set, test_set = train_test_split(dataset, test_size = testset_size, random_state = randomState)
  #print(('X_train samples : '), X_train[:1])
  #print(('X_test samples : '), X_test[0:1])
  #print(('y_train samples : '), y_train[0:20])
  #print(('y_test samples : '), y_test[0:20])
  
  return X_train, X_test, y_train, y_test

def confusion_matrix_function(Ytest,yPred):
  #from sklearn.metrics import confusion_matrix
  cm = confusion_matrix(Ytest,yPred)
  accuracy = float(cm.diagonal().sum())/len(Ytest)
  print("\nAccuracy Of SVM For The Given Dataset : ", accuracy)



In [ ]:
#X is the dataset and y is the labels
X_train, X_test, y_train, y_test= data_splitting(dataset_1)

In [ ]:
from sklearn.svm import SVC
result={}
for i in ['linear','poly','rbf']:
  print(i)
  classifier = SVC(kernel=i, random_state = 1,gamma='auto')
  classifier.fit(X_train,y_train)
  print('fit done')
  result[i]=classifier.predict(X_test)
  print('predict done')

print(result)

linear
